In [55]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', 15, 'max_rows', 30, 'display.precision', 4)

In [56]:
prices_name = 'price2302_df'
sentiments_name = 'sent2302_df'

In [57]:
prices = pd.read_csv(f'../cookbook_data/owndata/{prices_name}.csv', index_col=0)
sentiments = pd.read_csv(f'../cookbook_data/owndata/{sentiments_name}.csv', index_col=0)

In [58]:
prices.rename(columns={'Avg. Short Price /  Distance From Price': 'Short_Distance',
                       'Avg. Long Price /  Distance From Price': 'Long_Distance',
                       'Current Price': 'Current_Price'}, inplace=True)

In [59]:
distance = prices.iloc[:, 0:2].copy()

In [60]:
distance.Short_Distance = distance.Short_Distance.map(lambda x: x.rstrip(' pips'))
distance.Long_Distance = distance.Long_Distance.map(lambda x: x.rstrip(' pips'))

In [61]:
temp_short = distance.Short_Distance.str.split(
    ' ', expand=True).iloc[:, [0, 2]].copy()
temp_short.rename(columns={
                  temp_short.columns[0]: 'Short_Price', temp_short.columns[1]: 'Short_Distance'}, inplace=True)
# temp_short


In [62]:
temp_long = distance.Long_Distance.str.split(
    ' ', expand=True).iloc[:, [0, 2]].copy()
temp_long.rename(columns={
    temp_long.columns[0]: 'Long_Price', temp_long.columns[1]: 'Long_Distance'}, inplace=True)
# temp_long


In [63]:
result = pd.concat([temp_short, temp_long, prices.Current_Price], axis=1)
# result
# result.to_csv('../cookbook_data/owndata/prices_cleandf.csv')

In [64]:
# result.dtypes.value_counts()

In [65]:
result = result.apply(pd.to_numeric)
# result.dtypes

In [66]:
# result.select_dtypes(include=['int'])

In [67]:
# result.select_dtypes(include=['float'])

In [68]:
# result.select_dtypes(include=['number'])

##### addition functions

In [69]:
def max_streak(s):
    s1 = s.cumsum()
    return s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1, fill_value=0).max()

In [70]:
# weekly weight loss updates
def find_perc_loss(s):
    ''' calculate percent loss in weight '''
    return (s - s.iloc[0]) / s.iloc[0]


In [71]:
# generic closure to build all of our customized functions
def make_agg_func(func, name, *args, **kwargs):
    def wrapper(x):
        # print(type(x))    # x ~ s: Series performs function
        return func(x, *args, **kwargs)
    wrapper.__name__ = name
    return wrapper

# my_agg1 = make_agg_func(pct_between, 'pct_1_3k', low=1000, high=3000)
# my_agg2 = make_agg_func(pct_between, 'pct_10_30k', 10000, 30000)

In [72]:
def pct_between(s, low, high):
    return s.between(low, high).mean()

In [73]:
def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

In [74]:
def test_return_normality(stock_data):
    close = stock_data['Close']
    daily_return = close.pct_change().dropna()
    daily_return.hist(bins=20)
    mean = daily_return.mean()
    std = daily_return.std()

    abs_z_score = abs(daily_return - mean) / std

    pcts = [abs_z_score.lt(i).mean() for i in range(1, 4)]
    for count, pct in enumerate(pcts, 1):
        print(f'{pct:.3f} fall within {count}')

##### end add function

In [75]:
# result.index

In [76]:
result['Relative_SL'] = result.Short_Distance / result.Long_Distance
result['RelativePrice_SL'] = result.Short_Price / result.Long_Price

In [77]:
result.sort_values(by='Short_Distance', ascending=False, key=abs).head()

,Short_Price,Short_Distance,Long_Price,Long_Distance,Current_Price,Relative_SL,RelativePrice_SL
Symbol,,,,,,,
XAUUSD,1855.5271,-5402,1876.6233,3293,1909.5500,-1.6404,0.9888
USDCNH,6.4338,924,6.4008,-594,6.3414,-1.5556,1.0052
AUDJPY,78.9229,-373,83.0700,-42,82.6540,8.8810,0.9501
GBPJPY,152.2916,-368,155.5246,44,155.9690,-8.3636,0.9792
CHFJPY,121.8803,-338,125.0950,17,125.2650,-19.8824,0.9743


In [78]:
result.sort_values(by='Long_Distance', ascending=False, key=abs).head()

,Short_Price,Short_Distance,Long_Price,Long_Distance,Current_Price,Relative_SL,RelativePrice_SL
Symbol,,,,,,,
XAUUSD,1855.5271,-5402,1876.6233,3293,1909.5500,-1.6404,0.9888
USDCNH,6.4338,924,6.4008,-594,6.3414,-1.5556,1.0052
EURAUD,1.5677,-16,1.5919,-226,1.5693,0.0708,0.9848
EURCAD,1.4373,-39,1.4612,-200,1.4412,0.1950,0.9836
AUDCHF,0.6535,-62,0.6769,-172,0.6597,0.3605,0.9654


In [79]:
result.sort_values(by='Relative_SL', ascending=False, key=abs).head()

,Short_Price,Short_Distance,Long_Price,Long_Distance,Current_Price,Relative_SL,RelativePrice_SL
Symbol,,,,,,,
CHFJPY,121.8803,-338,125.0950,17,125.2650,-19.8824,0.9743
USDJPY,112.0758,-267,114.8845,-14,114.7430,19.0714,0.9756
GBPCAD,1.7240,-91,1.7337,-6,1.7331,15.1667,0.9944
AUDJPY,78.9229,-373,83.0700,-42,82.6540,8.8810,0.9501
GBPJPY,152.2916,-368,155.5246,44,155.9690,-8.3636,0.9792


In [80]:
result.sort_values(by='Relative_SL', ascending=True, key=abs).head()

,Short_Price,Short_Distance,Long_Price,Long_Distance,Current_Price,Relative_SL,RelativePrice_SL
Symbol,,,,,,,
USDCHF,0.9159,0,0.9244,-85,0.9159,-0.0000,0.9908
EURAUD,1.5677,-16,1.5919,-226,1.5693,0.0708,0.9848
NZDCHF,0.6127,-22,0.6290,-141,0.6149,0.1560,0.9741
EURUSD,1.1276,-28,1.1448,-144,1.1304,0.1944,0.9850
EURCAD,1.4373,-39,1.4612,-200,1.4412,0.1950,0.9836


In [81]:
# balance or not balance
result.sort_values(by='RelativePrice_SL', ascending=False).head()

,Short_Price,Short_Distance,Long_Price,Long_Distance,Current_Price,Relative_SL,RelativePrice_SL
Symbol,,,,,,,
USDCNH,6.4338,924,6.4008,-594,6.3414,-1.5556,1.0052
GBPCAD,1.7240,-91,1.7337,-6,1.7331,15.1667,0.9944
GBPNZD,2.0034,-211,2.0202,43,2.0245,-4.9070,0.9917
USDCHF,0.9159,0,0.9244,-85,0.9159,-0.0000,0.9908
EURCHF,1.0410,56,1.0513,-159,1.0354,-0.3522,0.9902


In [82]:
result.sort_values(by='RelativePrice_SL', ascending=True).head()

,Short_Price,Short_Distance,Long_Price,Long_Distance,Current_Price,Relative_SL,RelativePrice_SL
Symbol,,,,,,,
AUDJPY,78.9229,-373,83.0700,-42,82.6540,8.8810,0.9501
XAGUSD,23.5857,-61,24.5900,-40,24.1920,1.5250,0.9592
AUDCHF,0.6535,-62,0.6769,-172,0.6597,0.3605,0.9654
NZDUSD,0.6645,-70,0.6842,-127,0.6715,0.5512,0.9712
NZDCHF,0.6127,-22,0.6290,-141,0.6149,0.1560,0.9741


In [83]:
# sorted_result.head()

In [84]:
# sorted_result.tail()

In [85]:
# result.select_dtypes(include=['string'])

In [86]:
# need update current price
# result.filter(items=['Short_Price', 'Long_Price', 'Current_Price']).head()
result.filter(items=['Short_Price', 'Long_Price']).head()

,Short_Price,Long_Price
Symbol,,
EURUSD,1.1276,1.1448
GBPUSD,1.3479,1.3645
USDJPY,112.0758,114.8845
GBPJPY,152.2916,155.5246
USDCAD,1.2679,1.2871


In [87]:
result.filter(like='Distance').head()

,Short_Distance,Long_Distance
Symbol,,
EURUSD,-28,-144
GBPUSD,-115,-51
USDJPY,-267,-14
GBPJPY,-368,44
USDCAD,-70,-122


In [88]:
# result.filter(regex='\d').head()
# result.columns

In [89]:
# result.values

In [90]:
# result.count()

In [91]:
# result.min()

In [92]:
# result.max()

In [93]:
# result.describe()

In [94]:
# pd.set_option('max_rows', 20)
# result.describe(percentiles=[.01, .05, .1, .3, .4, .5, .6, .7, .8, .9, .95, .99])

In [95]:
# pd.set_option('max_rows', 10)
# result.isnull().sum()

In [96]:
# result.min(skipna=False)

In [97]:
# result.isnull().sum().sum()

In [98]:
# result.isnull().any()

In [99]:
# result.isnull().dtypes

In [100]:
# result.dtypes
# result.select_dtypes(['int']).fillna('').max()
result.select_dtypes(['int']).fillna('').sort_values(by=['Short_Distance'], ascending=False).tail(10)   # head

,Short_Distance,Long_Distance
Symbol,,
GBPUSD,-115,-51
EURNZD,-134,-155
GBPNZD,-211,43
GBPAUD,-217,-133
EURJPY,-223,-77
USDJPY,-267,-14
CHFJPY,-338,17
GBPJPY,-368,44
AUDJPY,-373,-42


In [101]:
result.Long_Distance.sort_values(ascending=False).head(10)              

Symbol
XAUUSD    3293
GBPJPY      44
GBPNZD      43
CHFJPY      17
GBPCAD      -6
USDJPY     -14
AUDNZD     -19
XAGUSD     -40
AUDJPY     -42
GBPUSD     -51
Name: Long_Distance, dtype: int64

#### update day by day this price

In [102]:
# result

In [103]:
# sentiments = sentiments.apply(pd.to_numeric)

In [104]:
sentiments.dtypes

Action        object
Percentage    object
Volume        object
Positions      int64
dtype: object

In [105]:
sentiments.head(10)

,Action,Percentage,Volume,Positions
Symbol,,,,
EURCHF,Long,96%,4689.19 lots,10356
EURGBP,Long,93%,3895.74 lots,14164
NZDCHF,Long,91%,356.07 lots,2198
NZDCHF,Short,9%,35.29 lots,505
USDCNH,Long,88%,4.23 lots,41
USDCHF,Long,88%,3901.09 lots,13785
AUDNZD,Short,87%,2168.35 lots,6864
EURAUD,Long,87%,1424.50 lots,4846
GBPCAD,Short,83%,1262.06 lots,4478


In [106]:
sentiments.tail(10)

,Action,Percentage,Volume,Positions
Symbol,,,,
NZDCAD,Long,25%,383.57 lots,1575
AUDCHF,Short,22%,196.10 lots,1148
XAUUSD,Long,21%,1161.67 lots,5384
CHFJPY,Long,19%,251.22 lots,977
GBPNZD,Long,18%,257.07 lots,1493
GBPCAD,Long,17%,265.77 lots,1078
EURAUD,Short,13%,219.58 lots,1014
AUDNZD,Long,13%,330.84 lots,2055
USDCHF,Short,12%,514.21 lots,3008


## First Example

In [107]:
status = sentiments.iloc[:, 0].copy()
# status

In [108]:
details_copy = sentiments.iloc[:, 1:4].copy()
details_copy.Volume = details_copy.Volume.map(lambda x: x.rstrip(' lots'))
details_copy.Percentage = details_copy.Percentage.map(lambda x: x.rstrip('%'))
details_copy = details_copy.apply(pd.to_numeric)

details = pd.concat([status, details_copy], axis=1)

details.head(10)

,Action,Percentage,Volume,Positions
Symbol,,,,
EURCHF,Long,96,4689.19,10356
EURGBP,Long,93,3895.74,14164
NZDCHF,Long,91,356.07,2198
NZDCHF,Short,9,35.29,505
USDCNH,Long,88,4.23,41
USDCHF,Long,88,3901.09,13785
AUDNZD,Short,87,2168.35,6864
EURAUD,Long,87,1424.50,4846
GBPCAD,Short,83,1262.06,4478


In [109]:
# details.sort_values(by=['Volume'], ascending=False).head(10)   # head

In [110]:
details.sort_values(by=['Percentage'], ascending=False).head(10)

,Action,Percentage,Volume,Positions
Symbol,,,,
EURCHF,Long,96,4689.19,10356
EURGBP,Long,93,3895.74,14164
NZDCHF,Long,91,356.07,2198
USDCNH,Long,88,4.23,41
USDCHF,Long,88,3901.09,13785
AUDNZD,Short,87,2168.35,6864
EURAUD,Long,87,1424.50,4846
GBPCAD,Short,83,1262.06,4478
GBPNZD,Short,82,1207.59,4786


In [111]:
# details.sort_values(by=['Percentage'], ascending=False).tail(10)

In [112]:
quote = r'GBP'
quote_filter = details[details.index.str.contains(quote + r'(?!$)')]
quote_filter.dtypes
# quote_filter = quote_filter.apply(pd.to_numeric)

Action         object
Percentage      int64
Volume        float64
Positions       int64
dtype: object

In [113]:
quote_filter.sort_values(by=['Percentage'], ascending=False)

,Action,Percentage,Volume,Positions
Symbol,,,,
GBPCAD,Short,83,1262.06,4478
GBPNZD,Short,82,1207.59,4786
GBPJPY,Short,66,1138.81,4875
GBPUSD,Short,65,5679.22,19809
GBPCHF,Long,61,424.38,2338
GBPAUD,Long,50,266.77,1622
GBPAUD,Short,50,271.33,1704
GBPCHF,Short,39,271.52,1145
GBPUSD,Long,35,3065.69,13351


In [114]:
quote = r'CHF'
quote_filter = details[details.index.str.contains(quote + r'(?!$)')]
quote_filter.dtypes
# quote_filter = quote_filter.apply(pd.to_numeric)

Action         object
Percentage      int64
Volume        float64
Positions       int64
dtype: object

In [115]:
quote_filter.sort_values(by=['Percentage'], ascending=False)

,Action,Percentage,Volume,Positions
Symbol,,,,
CHFJPY,Short,81,1039.15,3854
CHFJPY,Long,19,251.22,977


## More from sentiments

In [116]:
sentiments.Volume = sentiments.Volume.map(lambda x: x.rstrip(' lots'))
sentiments.Percentage = sentiments.Percentage.map(lambda x: x.rstrip('%'))

In [117]:
sentiments

,Action,Percentage,Volume,Positions
Symbol,,,,
EURCHF,Long,96,4689.19,10356
EURGBP,Long,93,3895.74,14164
NZDCHF,Long,91,356.07,2198
NZDCHF,Short,9,35.29,505
USDCNH,Long,88,4.23,41
...,...,...,...,...
GBPCAD,Long,17,265.77,1078
EURAUD,Short,13,219.58,1014
AUDNZD,Long,13,330.84,2055


In [118]:
numeric_sentiments = sentiments.iloc[:, 1:4].apply(pd.to_numeric)

In [119]:
result_sentiments = pd.concat([sentiments.Action, numeric_sentiments], axis=1)

In [120]:
result_sentiments
# result_sentiments.to_csv('../cookbook_data/owndata/sentiments_df.csv')

,Action,Percentage,Volume,Positions
Symbol,,,,
EURCHF,Long,96,4689.19,10356
EURGBP,Long,93,3895.74,14164
NZDCHF,Long,91,356.07,2198
NZDCHF,Short,9,35.29,505
USDCNH,Long,88,4.23,41
...,...,...,...,...
GBPCAD,Long,17,265.77,1078
EURAUD,Short,13,219.58,1014
AUDNZD,Long,13,330.84,2055


In [121]:
result_sentiments.sort_values(by=['Volume'], ascending=False).head(10)

,Action,Percentage,Volume,Positions
Symbol,,,,
EURUSD,Long,65,12536.95,38290
EURUSD,Short,35,6643.52,18856
GBPUSD,Short,65,5679.22,19809
EURCHF,Long,96,4689.19,10356
XAUUSD,Short,79,4441.29,21861
USDCHF,Long,88,3901.09,13785
EURGBP,Long,93,3895.74,14164
USDJPY,Long,52,3792.39,11322
EURJPY,Long,73,3646.19,10339


In [122]:
result_sentiments.sort_values(by=['Positions'], ascending=False).head(10)

,Action,Percentage,Volume,Positions
Symbol,,,,
EURUSD,Long,65,12536.95,38290
XAUUSD,Short,79,4441.29,21861
GBPUSD,Short,65,5679.22,19809
EURUSD,Short,35,6643.52,18856
EURGBP,Long,93,3895.74,14164
USDCHF,Long,88,3901.09,13785
GBPUSD,Long,35,3065.69,13351
USDCAD,Short,64,3019.32,11342
USDJPY,Long,52,3792.39,11322


#### Sum Positions by Symbol

#### Diff Percentage|Positions|Volume by Symbol

#### Merge Symbols and Actions

#### Compare Volume betwwen Quotes (with same Base): ... 

In [123]:
economic_calendar_1 = pd.read_csv('../cookbook_data/owndata/economic_calendar_1.csv', index_col=1)
economic_calendar_1.drop('id', axis=1, inplace=True)
time_format = '%d/%m/%Y'
economic_calendar_1.index = pd.to_datetime(economic_calendar_1.index, format=time_format)

In [124]:
notnan_economic_filter = economic_calendar_1[economic_calendar_1.currency.notnull()]
# notnan_economic_filter

In [125]:
pd.set_option('max_columns', 20, 'max_rows', 20)

quote_filter = notnan_economic_filter[notnan_economic_filter.currency.str.contains('JPY')]
quote_filter = quote_filter.dropna()

In [126]:
def strip_shit(df, cols=['actual', 'forecast', 'previous']):
    for col in cols:
        df[col] = df[col].map(lambda x: x.rstrip(
            '%').rstrip('T').rstrip('B').rstrip('K').rstrip('M'))
    return df

In [127]:
quote_filter.iloc[:, 5:8] = strip_shit(quote_filter.iloc[:, 5:8].copy())
quote_result_num = quote_filter.iloc[:, 5:8].apply(pd.to_numeric)
quote_results = pd.concat([quote_filter.iloc[:, :5], quote_result_num], axis=1)
quote_results.dtypes

time           object
zone           object
currency       object
importance     object
event          object
actual        float64
forecast      float64
previous      float64
dtype: object

In [128]:
quote_results.select_dtypes(include=['float']).head()

,actual,forecast,previous
date,,,
2022-01-06,52.10,51.10,53.00
2022-01-07,-1.30,1.60,-0.60
2022-01-07,-1.20,1.20,3.40
2022-01-07,0.50,0.50,0.30
2022-01-12,1.37,1.05,1.03


In [129]:
quote_results.select_dtypes(include=['object']).head()

,time,zone,currency,importance,event
date,,,,,
2022-01-06,07:30,japan,JPY,medium,Services PMI (Dec)
2022-01-07,06:30,japan,JPY,medium,Household Spending (YoY) (Nov)
2022-01-07,06:30,japan,JPY,medium,Household Spending (MoM) (Nov)
2022-01-07,06:30,japan,JPY,medium,Tokyo Core CPI (YoY) (Dec)
2022-01-12,06:50,japan,JPY,medium,Adjusted Current Account


#### NLP some event words: YoY, MoM
- Calculate months range, monthly change, yearly changes
- Remove shitty words such as Q1

#### [About Fx Market Holiday](https://www.forex.academy/how-do-public-holidays-affect-the-forex-market/)

In [130]:
economic_calendar_1

,time,zone,currency,importance,event,actual,forecast,previous
date,,,,,,,,
2022-01-01,All Day,japan,NaN,NaN,Japan - New Year's Day,NaN,NaN,NaN
2022-01-01,01:00,united states,USD,medium,U.S. Baker Hughes Oil Rig Count,480,NaN,480
2022-01-01,01:00,united states,USD,medium,U.S. Baker Hughes Total Rig Count,586,NaN,586
2022-01-02,All Day,japan,NaN,NaN,Japan - Market Holiday,NaN,NaN,NaN
2022-01-03,All Day,united kingdom,NaN,NaN,United Kingdom - New Year's Day,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-01-10,All Day,japan,NaN,NaN,Japan - Respect for the Aged Day,NaN,NaN,NaN
2022-01-17,All Day,united states,NaN,NaN,"United States - Martin Luther King, Jr. Day",NaN,NaN,NaN
2022-01-26,All Day,australia,NaN,NaN,Australia - Australia Day,NaN,NaN,NaN


In [131]:
# first way to filter holiday 
holiday_filter = economic_calendar_1[economic_calendar_1.currency.isnull()]

# holiday_filter.to_csv('../cookbook_data/owndata/economic_calendar_1_holiday.csv')
holiday_filter

,time,zone,currency,importance,event,actual,forecast,previous
date,,,,,,,,
2022-01-01,All Day,japan,NaN,NaN,Japan - New Year's Day,NaN,NaN,NaN
2022-01-02,All Day,japan,NaN,NaN,Japan - Market Holiday,NaN,NaN,NaN
2022-01-03,All Day,united kingdom,NaN,NaN,United Kingdom - New Year's Day,NaN,NaN,NaN
2022-01-03,All Day,australia,NaN,NaN,Australia - New Year's Day,NaN,NaN,NaN
2022-01-03,All Day,canada,NaN,NaN,Canada - New Year's Day,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-01-10,All Day,japan,NaN,NaN,Japan - Respect for the Aged Day,NaN,NaN,NaN
2022-01-17,All Day,united states,NaN,NaN,"United States - Martin Luther King, Jr. Day",NaN,NaN,NaN
2022-01-26,All Day,australia,NaN,NaN,Australia - Australia Day,NaN,NaN,NaN


In [132]:
len(economic_calendar_1), len(holiday_filter), len(notnan_economic_filter)

(426, 45, 381)

#### events detail: (Split to Nation)
- Housing: HPI / Building Approvals
- Retail Sales
- CPI / HIPC / Core CPI
- PMI: Services / Markit Composite / Manufacturing / Composite 
- Unemployment Change / Unemployment Rate
- Speaks: RBA
- Business Confidence
- Trade Balance
- Inflation Report
- MPC: Vote Cut

#### GBP Examples
- Housing: [HPI](https://www.investing.com/economic-calendar/nationwide-hpi-850), [Halifax](https://www.investing.com/economic-calendar/halifax-house-price-index-844) House Price Index, [RICS](https://www.investing.com/economic-calendar/rics-house-price-balance-264) House Price Balance
- PMI: [Manufacturing](https://www.investing.com/economic-calendar/manufacturing-pmi-204) ... [Composite](https://www.investing.com/economic-calendar/composite-pmi-1934), [Services](https://www.investing.com/economic-calendar/services-pmi-274), [Construction](https://www.investing.com/economic-calendar/construction-pmi-44)
- [Inflation](https://www.investing.com/economic-calendar/boe-inflation-report-15) 
- [PPI](https://www.investing.com/economic-calendar/ppi-input-242)
- MPC, BOE Member Speech
- [Interest Rate](https://www.investing.com/economic-calendar/interest-rate-decision-170) Decision
- Governor Speaks
- [CFTC](https://www.investing.com/economic-calendar/cftc-gbp-speculative-positions-1612) GBP speculative net positions
- GBP/USD [Index](https://www.investing.com/economic-calendar/investing.com-gbp-usd-index-1156)
- [BRC](https://www.investing.com/economic-calendar/brc-retail-sales-monitor-18) [Retail Sales](https://www.investing.com/economic-calendar/retail-sales-731), [Core](https://www.investing.com/economic-calendar/core-retail-sales-857)
- [Business Investment](https://www.investing.com/economic-calendar/business-investment-30)
- [GDP](https://www.investing.com/economic-calendar/gdp-121), [Monthly](https://www.investing.com/economic-calendar/monthly-gdp-3m-3m-change-1800)
- Production [Industrial](https://www.investing.com/economic-calendar/industrial-production-158), [Manufacturing](https://www.investing.com/economic-calendar/manufacturing-production-205)
- [Trade Balance](https://www.investing.com/economic-calendar/trade-balance-284), [Non-EU](https://www.investing.com/economic-calendar/trade-balance-non-eu-849)
- [NIESR](https://www.investing.com/economic-calendar/niesr-monthly-gdp-tracker-1797)
- [Average Earnings Index +Bonus](https://www.investing.com/economic-calendar/average-earnings-index-bonus-7)
- [Claimant Count Change](https://www.investing.com/economic-calendar/claimant-count-change-39)
- [Employment](https://www.investing.com/economic-calendar/employment-change-3m-3m-1743), [Unemployment](https://www.investing.com/economic-calendar/unemployment-rate-297)
- [CBI](https://www.investing.com/economic-calendar/cbi-industrial-trends-orders-34) Industrial Trends Orders

In [133]:
economic_calendar_2 = pd.read_csv('../cookbook_data/owndata/economic_calendar_2.csv', index_col=1)
economic_calendar_2.drop('id', axis=1, inplace=True)

time_format = '%d/%m/%Y'
economic_calendar_2.index = pd.to_datetime(economic_calendar_2.index, format=time_format)

In [134]:
def quote_filtering(df, quote):
    pd.set_option('max_columns', 20, 'max_rows', 20)

    quote_filter = df[df.currency.str.contains(quote)]
    quote_filter_medium = quote_filter[quote_filter.importance.str.contains('medium')]
    quote_filter_high = quote_filter[quote_filter.importance.str.contains('high')]
    return quote_filter_medium, quote_filter_high

In [135]:
def quote_numericing(df):
    df.iloc[:, 5:8] = strip_shit(df.iloc[:, 5:8].copy())
    quote_result_num = df.iloc[:, 5:8].apply(pd.to_numeric)
    return pd.concat([df.iloc[:, :5], quote_result_num], axis=1)


In [136]:
def split_filter(df):
    return df[df.isna().any(axis=1)], quote_numericing(df.copy().dropna())


In [137]:
notnan_economic_filter_2 = economic_calendar_2[economic_calendar_2.currency.notnull()]

In [138]:
quote_filter_medium, quote_filter_high = quote_filtering(notnan_economic_filter_2, 'USD')
quote_notall_num, result_numeric = split_filter(quote_filter_high)

In [139]:
result_numeric

,time,zone,currency,importance,event,actual,forecast,previous
date,,,,,,,,
2022-02-01,22:00,united states,USD,high,ISM Manufacturing PMI (Jan),57.600,57.500,58.800
2022-02-01,22:00,united states,USD,high,JOLTs Job Openings (Dec),10.925,10.300,10.775
2022-02-02,20:15,united states,USD,high,ADP Nonfarm Employment Change (Jan),-301.000,207.000,776.000
2022-02-02,22:30,united states,USD,high,Crude Oil Inventories,-1.046,1.525,2.377
2022-02-03,20:30,united states,USD,high,Initial Jobless Claims,238.000,245.000,261.000
2022-02-03,22:00,united states,USD,high,ISM Non-Manufacturing PMI (Jan),59.900,59.500,62.300
2022-02-04,20:30,united states,USD,high,Nonfarm Payrolls (Jan),467.000,150.000,510.000
2022-02-04,20:30,united states,USD,high,Unemployment Rate (Jan),4.000,3.900,3.900


#### dropna make some case not removing cell number Unit 

In [140]:
quote_notall_num

,time,zone,currency,importance,event,actual,forecast,previous
date,,,,,,,,
2022-02-04,22:45,united states,USD,high,U.S. President Biden Speaks,NaN,NaN,NaN
2022-02-09,00:00,united states,USD,high,EIA Short-Term Energy Outlook,NaN,NaN,NaN
2022-02-09,22:30,united states,USD,high,Crude Oil Inventories,NaN,0.675M,-1.046M
2022-02-10,20:30,united states,USD,high,Core CPI (MoM) (Jan),NaN,0.5%,0.6%
2022-02-10,20:30,united states,USD,high,Initial Jobless Claims,NaN,230K,238K
2022-02-11,22:00,united states,USD,high,Fed Monetary Policy Report,NaN,NaN,NaN
2022-02-15,20:30,united states,USD,high,PPI (MoM) (Jan),NaN,0.4%,0.2%
2022-02-16,20:30,united states,USD,high,Core Retail Sales (MoM) (Jan),NaN,0.4%,-2.3%
2022-02-16,20:30,united states,USD,high,Retail Sales (MoM) (Jan),NaN,1.7%,-1.9%
